In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Dense, GRU, Input
from tensorflow.keras.models import Model
import os
import cv2
import numpy as np
import pandas as pd


In [6]:

# Model definition
def create_crnn(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Reshape for RNN layers
    x = Reshape((-1, x.shape[-1]))(x)

    # Recurrent layers
    x = GRU(128, return_sequences=True)(x)
    x = GRU(128, return_sequences=True)(x)

    # Fully connected layer
    outputs = Dense(num_classes + 1, activation='softmax')(x)


    model = Model(inputs, outputs)
    return model

In [4]:
from tensorflow.keras.utils import to_categorical

In [7]:

# Define character set (modify if needed)
CHARACTERS = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789()"

def encode_label(label):
    return [CHARACTERS.index(char) for char in label]

def load_data(mapping_file, image_size):
    images, labels = [], []
    with open(mapping_file, 'r') as file:
        for line in file:
            image_path, label = line.strip().split()
            # Load and preprocess image
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, image_size) / 255.0
            image = np.expand_dims(image, axis=-1)
            images.append(image)
            labels.append(encode_label(label))
    return np.array(images), labels

# Example usage
mapping_file = "dataset/labels.txt"
image_size = (128, 32)  # Resize all images to 128x32
X, y = load_data(mapping_file, image_size)

In [8]:
from tensorflow.keras.layers import Lambda
from tensorflow.keras.backend import ctc_batch_cost
from tensorflow.keras.optimizers import Adam

In [9]:

# Define input shape and number of classes
input_shape = (32, 128, 1)  # Image dimensions (height, width, channels)
num_classes = len(CHARACTERS)

# Create the CRNN model
model = create_crnn(input_shape, num_classes)

def ctc_loss(y_true, y_pred):
    # Get the batch size
    batch_size = tf.shape(y_pred)[0]
    # Predicted sequence lengths (based on model output width)
    input_length = tf.fill([batch_size], tf.shape(y_pred)[1])
    # Actual label lengths (non-padded parts of y_true)
    label_length = tf.reduce_sum(tf.cast(tf.not_equal(y_true, -1), tf.int32), axis=1)
    return ctc_batch_cost(y_true, y_pred, input_length, label_length)


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=ctc_loss)

# Convert labels to categorical and pad them
from tensorflow.keras.preprocessing.sequence import pad_sequences
y_padded = pad_sequences(y, padding='post', value=-1)  # Use -1 as padding value

# Train the model
model.fit(X, y_padded, batch_size=32, epochs=50, validation_split=0.1)

AttributeError: 'Functional' object has no attribute 'display'